# **HW2 : Decision Tree and Random Forest**
In *assignment 2*, you need to finish :

1. Basic Part : Implement a **Decision Tree** model and predict whether the patients in the validation set have diabetes
> * Step 1 : Load the input data
> * Step 2 : Calculate the Entropy and Information Gain
> * Step 3 : Find the Best Split
> * Step 4 : Split into 2 branches
> * Step 5 : Build decision tree
> * Step 6 : Save the answers from step2 to step5
> * Step 7 : Split data into training set and validation set
> * Step 8 : Train a decision tree model with training set
> * Step 9 : Predict the cases in the *validation set* by using the model trained in *Step8*
> * Step 10 : Calculate the f1-score of your predictions in *Step9*
> * Step 11 : Write the Output File

2. Advanced Part : Build a **Random Forest** model to make predictions
> * Step 1 : Load the input data
> * Step 2 : Load the test data
> * Step 3 : Build a random forest
> * Step 4 : Predict the cases in the test data by using the model trained in *Step3*
> * Step 5 : Save the predictions(from *Step 4*) in a csv file



# **Basic Part** (60%)
In this part, your need to implement a Decision Tree model by completing the following given functions.

Also, you need to run these functions with the given input variables and save the output in a csv file **hw2_basic.csv**

## Import Packages


> Note : You **cannot** import any other packages in both basic part and advanced part






In [2322]:
import numpy as np
import pandas as pd
import math
import random
from numpy import sqrt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

## Step1: Load the input data
First, load the input file **hw2_input_basic.csv**

In [2323]:
input_data = pd.read_csv('hw2_input_basic.csv')

## Global attributes
Define the global attributes
> Note : You **cannot** modify the values of these attributes we given in the basic part

In [2324]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = input_data.shape[1] - 1

> You can add your own global attributes here

In [2325]:
pd.options.mode.chained_assignment = None 

## Step2 : Calculate the Entropy and Information Gain 
Calculate the information gain and entropy values before separate data into left subtree and right subtree

In [2326]:
from math import log2

def entropy(data):
  """
  This function measures the amount of uncertainty in a probability distribution
  args: 
  * data(type: DataFrame): the data you're calculating for the entropy
  return:
  * entropy_value(type: float): the data's entropy
  """

  total = data.shape[0]
  pos_case = data['diabetes_mellitus'].sum()
  
  if pos_case == 0:
    e = 0
  elif pos_case == total:
    e = 0
  else:
    p = pos_case/total
    e = -p*log2(p)-(1-p)*log2(1-p)

  entropy_value = round(e,5)
  
  return entropy_value

# [Note] You have to save the value of "ans_entropy" into the output file
ans_entropy = entropy(input_data)
print("ans_entropy = ", ans_entropy)

ans_entropy =  0.98714


In [2327]:
def information_gain(data, mask):
  """
  This function will calculate the information gain
  args:
  * data(type: DataFrame): the data you're calculating for the information gain
  * mask(type: Series): partition information(left/right) of current input data, 
    - boolean 1(True) represents split to left subtree
    - boolean 0(False) represents split to right subtree
  return:
  * ig(type: float): the information gain you can obtain by classify data with this given mask
  """
  #Split data base on mask
  left_tree_true = data[mask]
  right_tree_false = data[~mask]

  #Calculate the number of cases in left and right subtree respectively
  n_left_tree = left_tree_true.shape[0]
  n_right_tree = right_tree_false.shape[0]
  n_total = data.shape[0]

  #Calculate the weight of left and right subtree 
  weight_left = n_left_tree/n_total
  weight_right = n_right_tree/n_total

  #Claculate the entropy of left and right subtrees
  left_entropy = entropy(left_tree_true)
  right_entropy = entropy(right_tree_false)

  entropy_before = entropy(data)
  entropy_after = weight_left * left_entropy + weight_right * right_entropy
  
  ig = round(entropy_before - entropy_after,5)

  return ig

# [Note] You have to save the value of "ans_informationGain" into your output file
temp1 = np.zeros((int(input_data.shape[0]/4), 1), dtype=bool)
temp2 = np.ones(((input_data.shape[0]-int(input_data.shape[0]/4), 1)), dtype=bool)
temp_mask = np.concatenate((temp1, temp2))
df_mask = pd.DataFrame(temp_mask, columns=['mask'])
ans_informationGain = information_gain(input_data, df_mask['mask'])
print("ans_informationGain = ", ans_informationGain)

ans_informationGain =  0.08346


## Step3 : Find the Best Split
Find the best split combination, **feature** and **threshold**, by calculating the information gain


In [2328]:
def find_best_split(data):
  """
  This function will find the best split combination of data
  args:
  * data(type: DataFrame): the input data
  return
  * best_ig(type: float): the best information gain you obtain
  * best_threshold(type: float): the value that splits data into 2 branches
  * best_feature(type: string): the feature that splits data into 2 branches
  """

  n = n_features
  n_case = int(data.shape[0])
  df = data.iloc[0:n_case]
  features = df.columns

  step = 1

  table = np.zeros((3, n), dtype=float)

  mask_frag = np.ones((step, 1), dtype=bool)

  for i in range(n):
    mask = np.zeros((n_case, 1), dtype=bool)
    df['mask'] = mask
    for j in range(0,n_case-1,step):
      select = df.sort_values(by=[features[i]])
      mask[j:j+step] = mask_frag
      temp = select.T.columns
      index = int(temp[j+step-1])
      index2 = int(temp[j+step])
      a = select[features[i]][index]
      b = select[features[i]][index2]
      if a == b:
        continue
      select['mask'] = mask
      t_threshold = (select[features[i]][index]+select[features[i]][index2])/2
      select = select.sort_index()
      t_ig = information_gain(select, select['mask'])
      if table[0][i] < t_ig:
        table[0][i] = t_ig
        table[1][i] = t_threshold
        table[2][i] = i
  
  temp = pd.DataFrame(table)
  result = temp.sort_values(by=[0], ascending=False, axis=1).columns
  index = int(result[0])
  best_ig = temp[index][0]
  best_threshold = temp[index][1]
  best_feature = features[int(temp[index][2])]

  return best_ig, best_threshold, best_feature


# [Note] You have to save the value of "ans_ig", "ans_value", and "ans_name" into the output file
ans_ig, ans_value, ans_name = find_best_split(input_data)
print("ans_ig = ", ans_ig)
print("ans_value = ", ans_value)
print("ans_name = ", ans_name)

ans_ig =  0.35229
ans_value =  235.5
ans_name =  glucose_apache


## Step4 : Split into 2 branches
Using the best split combination you find in function *find_best_split()* to split data into Left Subtree and Right Subtree 

In [2329]:
def make_partition(data, feature, threshold):
  """
  This function will split the data into 2 branches
  args:
  * data(type: DataFrame): the input data
  * feature(type: string): the attribute(column name)
  * threshold(type: float): the threshold for splitting the data
  return:
  * left(type: DataFrame): the divided data that matches(less than or equal to) the assigned feature's threshold
  * right(type: DataFrame): the divided data that doesn't match the assigned feature's threshold
  """
  n = int(data.shape[0])
  mask = np.zeros((n, 1), dtype=bool)
  valid_index = data.T.columns
  j = 0
  for i in valid_index:
    mask[j] = (data[feature][i] <= threshold)
    j = j + 1
  

  left = data[mask]
  right = data[~mask]

  return left, right

# [Note] You have to save the value of "ans_left" into the output file
left, right = make_partition(input_data, 'age', 61.0)
ans_left = left.shape[0]
print("ans_left = ", ans_left)

ans_left =  10


## Step5 : Build Decision Tree
Use the above functions to implement the decision tree

Instructions: 
1.  If current depth < max_depth and the remaining number of samples > min_samples_split: continue to classify those samples
2.  Use function *find_best_split()* to find the best split combination
3.  If the obtained information gain is **greater than 0**: can build a deeper decision tree (add depth)
4. Use function *make_partition()* to split the data into two parts
5. Save the features and corresponding thresholds (starting from the root) used by the decision tree into *ans_features[]* and *ans_thresholds[]* respectively




In [2330]:
def build_tree(data, max_depth, min_samples_split, depth):
  """
  This function will build the decision tree
  args:
  * data(type: DataFrame): the data you want to apply to the decision tree
  * max_depth: the maximum depth of a decision tree
  * min_samples_split: the minimum number of instances required to do partition
  * depth: the height of the current decision tree
  return:
  * subtree: the decision tree structure including root, branch, and leaf (with the attributes and thresholds)
  """

  # check the condition of current depth and the remaining number of samples
  if depth < max_depth:
    # call find_best_split() to find the best combination
    ig, threshold, feature = find_best_split(data)
    # check the value of information gain is greater than 0 or not 
    if ig > 0:
      # update the depth
      depth = depth + 1
      # call make_partition() to split the data into two parts
      left_st, right_st = make_partition(data, feature, threshold)
      # If there is no data split to the left tree OR no data split to the left tree
      if left_st.empty:
        # return the label of the majority
        t = right_st.shape[0]
        p = right_st['diabetes_mellitus'].sum()
        n = t-p
        label = int(p > n)
        return label
      elif right_st.empty: 
        t = left_st.shape[0]
        p = left_st['diabetes_mellitus'].sum()
        n = t-p
        label = int(p > n)
        return label
      else:
        question = "{} {} {}".format(feature, "<=", threshold)
        subtree = {question: []}
        # call function build_tree() to recursively build the left subtree and right subtree
        left_subtree = build_tree(left_st, max_depth, min_samples_split, depth)
        right_subtree = build_tree(right_st, max_depth, min_samples_split, depth)
        if left_subtree == right_subtree:
          subtree = left_subtree
        else:
          subtree[question].append(left_subtree)
          subtree[question].append(right_subtree)
    else:
      # return the label of the majority
      t = data.shape[0]
      p = data['diabetes_mellitus'].sum()
      n = t-p
      label = int(p > n)
      return label
  else:
    # return the label of the majority
    t = data.shape[0]
    p = data['diabetes_mellitus'].sum()
    n = t-p
    label = int(p > n)
    return label

  return subtree

An example of the output from *build_tree()* 
```
{'bmi <= 33.5': [1, {'age <= 68.5': [0, 1]}]}
```
Therefore, 
```
ans_features = ['bmi', 'age']
ans_thresholds = [33.5, 68.5]
```



In [2331]:
ans_features = []
ans_thresholds = []

decisionTree = build_tree(input_data, max_depth, min_samples_split, depth)
decisionTree

{'glucose_apache <= 235.5': [{'heart_rate_apache <= 143.5': [0, 1]}, 1]}

In [2332]:
def PreOrderSearch(tree,feature_queue,thresholds_queue):
    
    feature_q = feature_queue
    thresholds_q = thresholds_queue

    for key in tree:
        feature_q.append(key.split(" <= ")[0])
        thresholds_q.append(key.split(" <= ")[1])

        if type(tree[key][0]) == dict:
            feature_q, thresholds_q = PreOrderSearch(tree[key][0], feature_q, thresholds_q)

        if type(tree[key][1]) == dict:
            feature_q, thresholds_q = PreOrderSearch(tree[key][1], feature_q, thresholds_q)
    
    return feature_q, thresholds_q

ans_features, ans_thresholds = PreOrderSearch(decisionTree,[],[])

# [Note] You have to save the features in the "decisionTree" structure (from root to branch and leaf) into the output file
ans_features

['glucose_apache', 'heart_rate_apache']

In [2333]:
# [Note] You have to save the corresponding thresholds for the features in the "ans_features" list into the output file
ans_thresholds

['235.5', '143.5']

## Step6 : Save answers

In [2334]:
basic = []
basic.append(ans_entropy)
basic.append(ans_informationGain)
basic.append(ans_ig)
basic.append(ans_value)
basic.append(ans_name)
basic.append(ans_left)
for i in range(len(ans_features)):
  basic.append(ans_features[i])
for m in range(len(ans_thresholds)):
  basic.append(ans_thresholds[m])

## Step7 : Split data
Split data into training set and validation set
> Note: We have split the data into training set and validation. You **cannot** change the distribution of the data.

In [2335]:
num_train = 20
num_validation = 10

training_data = input_data.iloc[:num_train]
validation_data = input_data.iloc[-num_validation:]

y_train = training_data[["diabetes_mellitus"]]
x_train = training_data.drop(['diabetes_mellitus'], axis=1)
y_validation = validation_data[["diabetes_mellitus"]]
x_validation = validation_data.drop(['diabetes_mellitus'], axis=1)
y_validation = y_validation.values.flatten()

print(input_data.shape)
print(training_data.shape)
print(validation_data.shape)

(30, 10)
(20, 10)
(10, 10)


## Step8 to Step10 : Make predictions with a decision tree

Define the attributions of the decision tree
> You **cannot** modify the values of these attributes in this part

In [2336]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = x_train.shape[1]

We have finished the function '*classify_data()*' below, however, you can modify this function if you prefer completing it on your own way.

In [2337]:
def classify_data(instance, tree):
  """
  This function will predict/classify the input instance
  args:
  * instance: a instance(case) to be predicted
  return:
  * answer: the prediction result (the classification result)
  """
  equation = list(tree.keys())[0]
  if equation.split()[1] == '<=':
    temp_feature = equation.split()[0]
    temp_threshold = equation.split()[2]
    if instance[temp_feature] > float(temp_threshold):
      answer = tree[equation][1]
    else:
      answer = tree[equation][0]
  else:
    if instance[equation.split()[0]] in (equation.split()[2]):
      answer = tree[equation][0]
    else:
      answer = tree[equation][1]

  if not isinstance(answer, dict):
    return answer
  else:
    return classify_data(instance, answer)


def make_prediction(tree, data):
  """
  This function will use your pre-trained decision tree to predict the labels of all instances in data
  args:
  * tree: the decision tree
  * data: the data to predict
  return:
  * y_prediction: the predictions
  """
  y_prediction = []
  
  valid_check = data.T.columns

  # [Note] You can call the function classify_data() to predict the label of each instance
  for i in list(valid_check):
    
    y_prediction.append(classify_data(data.loc[i], tree))
  
  return y_prediction

def calculate_score(y_true, y_pred):
  """
  This function will calculate the f1-score of the predictions
  args:
  * y_true: the ground truth
  * y_pred: the predictions
  return:
  * score: the f1-score
  """
  up = (y_true * y_pred).sum()
  
  down = y_true.sum() + np.array(y_pred).sum()

  score = round(2*up/down, 4)
  
  return score

In [2338]:
decision_tree = build_tree(training_data, max_depth, min_samples_split, depth)

y_pred = make_prediction(decision_tree, x_validation)

# [Note] You have to save the value of "ans_f1score" the your output file

ans_f1score = calculate_score(y_validation, y_pred)

print("ans_f1score = ", ans_f1score)

ans_f1score =  0.6667


## Step11 : Write the Output File
Save all of your answers in a csv file, named as **hw2_basic.csv**

In [2339]:
ans_path = 'hw2_basic.csv'

# [Note] You have to save the value of "ans_f1score" into the output file
basic.append(ans_f1score)
print(basic)

pd.DataFrame(basic).to_csv(ans_path, header = None, index = None)

[0.98714, 0.08346, 0.35229, 235.5, 'glucose_apache', 10, 'glucose_apache', 'heart_rate_apache', '235.5', '143.5', 0.6667]


# **Advanced Part** (35%)

## Step1: Load the input data
First, load the input file **hw2_input_advanced.csv**

In [2340]:
advanced_data = pd.read_csv('hw2_input_advanced.csv')
advanced_data

,age,bmi,gender,height,weight,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,...,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,diabetes_mellitus
0,72.0,35.027162,1,188.0,123.8,0.0,34.0,1.41,4.0,6.0,...,1.0,58.0,5.0,134.0,36.20,1.0,7.0,0.20,0.12,1
1,68.0,23.994027,1,180.3,78.0,0.0,19.0,1.56,4.0,6.0,...,0.0,130.0,26.0,139.0,35.90,0.0,18.7,0.05,0.02,1
2,54.0,29.566546,1,188.0,104.5,0.0,34.0,1.66,4.0,6.0,...,0.0,131.0,28.0,136.0,36.30,0.0,23.7,0.06,0.04,1
3,42.0,16.261908,1,182.9,54.4,0.0,30.0,1.21,4.0,6.0,...,0.0,137.0,8.0,135.0,36.30,0.0,7.1,0.01,0.00,1
4,82.0,24.017768,0,162.6,63.5,0.0,10.0,0.75,4.0,6.0,...,0.0,54.0,28.0,138.0,36.70,1.0,17.6,0.07,0.03,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8374,28.0,29.034281,1,185.4,99.8,0.0,11.0,0.90,4.0,6.0,...,0.0,114.0,37.0,136.0,37.05,0.0,17.2,0.01,0.00,0
8375,79.0,21.583026,0,157.0,53.2,0.0,17.0,0.71,4.0,6.0,...,0.0,50.0,24.0,147.0,36.80,0.0,5.4,0.04,0.02,1
8376,76.0,31.078941,0,167.6,87.3,1.0,33.0,4.62,4.0,6.0,...,0.0,44.0,25.0,143.0,36.00,0.0,6.3,0.08,0.02,1
8377,80.0,43.165799,1,182.9,144.4,0.0,27.0,1.53,4.0,6.0,...,0.0,139.0,41.0,136.0,36.50,0.0,7.2,0.10,0.05,0


You can split *advanced_data* into training set and validaiton set

In [2341]:
num_train = int(advanced_data.shape[0]*0.8)
num_validation = advanced_data.shape[0] - num_train

training_data = advanced_data.iloc[:num_train]
validation_data = advanced_data.iloc[-num_validation:]

y_train = training_data[["diabetes_mellitus"]]
x_train = training_data.drop(['diabetes_mellitus'], axis=1)
y_validation = validation_data[["diabetes_mellitus"]]
x_validation = validation_data.drop(['diabetes_mellitus'], axis=1)
y_validation = y_validation.values.flatten()

## Step2 : Load the test data
Load the input file **hw2_input_test.csv** to make predictions with the pre-trained random forest model

In [2342]:
x_test = pd.read_csv('hw2_input_test.csv')
x_test

,age,bmi,gender,height,weight,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,...,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,apache_4a_hospital_death_prob,apache_4a_icu_death_prob
0,62,32.866392,1,177.80,103.9,1,31.0,10.30,4,6,...,36.4,0,157,26,134,36.1,0,4.56,0.06,0.03
1,82,23.582766,0,157.50,58.5,0,26.0,0.54,3,4,...,32.8,0,42,25,142,36.1,0,6.00,0.14,0.06
2,61,31.684520,1,172.70,94.5,0,16.0,1.11,4,6,...,35.3,0,129,6,131,36.8,0,8.59,0.05,0.03
3,58,45.156250,0,160.00,115.6,0,19.0,0.70,1,4,...,30.1,1,131,23,138,34.9,1,16.03,0.33,0.22
4,74,25.817016,1,172.70,77.0,0,25.0,0.93,4,6,...,34.5,0,55,12,135,36.3,0,45.80,0.12,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,73,17.943584,0,157.48,44.5,0,12.0,0.30,4,6,...,33.8,0,129,9,144,36.9,0,7.70,0.02,0.01
836,79,29.049732,1,167.60,81.6,0,48.0,2.19,4,6,...,42.7,0,163,9,139,36.4,0,10.77,0.06,0.03
837,85,24.627827,0,152.40,57.2,0,11.0,0.48,3,5,...,29.5,0,67,9,139,36.6,0,7.35,0.16,0.05
838,68,32.510940,1,193.00,121.1,0,14.0,0.64,4,6,...,37.5,0,61,10,140,36.9,1,16.02,0.00,0.00


## Step3 : Build a Random Forest

Define the attributions of the random forest
> * You **can** modify the values of these attributes in advanced part
> * Each tree can have different attribute values
> * There must be **at least** 3 decision trees in the random forest model
> * Must use function *build_tree()* to build a random forest model
> * These are the parameters you can adjust : 


    ```
    max_depth = 
    depth = 0
    min_samples_split = 
    
    # total number of trees in a random forest
    n_trees = 

    # number of features to train a decision tree
    n_features = 

    # the ratio to select the number of instances
    sample_size = 
    n_samples = int(training_data.shape[0] * sample_size)
    ```




In [2343]:
# Define the attributes
max_depth = 4

depth = 0

min_samples_split = 2
    
# total number of trees in a random forest
n_trees = 4

# number of features to train a decision tree
n_features = 6

# the ratio to select the number of instances
sample_size = 0.25

n_samples = int(training_data.shape[0] * sample_size)

In [2344]:
def build_forest(data, n_trees, n_features, n_samples):
  """
  This function will build a random forest.
  args:
  * data: all data that can be used to train a random forest
  * n_trees: total number of tree
  * n_features: number of features
  * n_samples: number of instances
  return:
  * forest: a random forest with 'n_trees' of decision tree
  """
  forest = []
  # must reuse function build_tree()
  for j in range(n_trees):
    sel_data = data[j*n_samples:(j+1)*n_samples].T[j*n_features:(j+1)*n_features].T
    sel_data2 = data['diabetes_mellitus'][j*n_samples:(j+1)*n_samples]
    sel_data['diabetes_mellitus'] = sel_data2
    tree = build_tree(sel_data, max_depth, min_samples_split, depth)
    print(tree)
    forest.append(tree)
  return forest

In [2345]:
forest = build_forest(training_data, n_trees, n_features, n_samples)

KeyboardInterrupt: 

## Step4 : Make predictions with the random forest
> Note: Please print the f1-score of the predictions of each decision tree

In [ ]:
def make_prediction_forest(forest, data):
  """
  This function will use the pre-trained random forest to make the predictions
  args:
  * forest: the random forest
  * data: the data used to predict
  return:
  * y_prediction: the predicted results
  """


  return y_prediction

In [ ]:
y_pred_test = make_prediction_forest(forest, x_test)

NameError: name 'y_prediction' is not defined

## Step5 : Write the Output File
Save your predictions from the **random forest** in a csv file, named as **hw2_advanced.csv**

In [ ]:
advanced = []
for i in range(len(y_pred_test)):
  advanced.append(y_pred_test[i])

In [ ]:
advanced_path = 'hw2_advanced.csv'
pd.DataFrame(advanced).to_csv(advanced_path, header = None, index = None)